In [2]:
import pandas as pd
import popDMS
import numpy as np
import scipy.stats as st
AA  = ['A', 'R', 'N', 'D', 'C', 'E', 'Q', 'G', 'H', 'I','L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']


# PR8, Aichi68C, HIV Env BG505, HIV Env BF520

In [3]:
def preference_merge(REP, Target_name, count_file):
    pref_list = ['./data/prefs/'+Target_name+'-1_prefs.csv',
                 './data/prefs/'+Target_name+'-2_prefs.csv',
                 './data/prefs/'+Target_name+'-3_prefs.csv']

    df_count = pd.read_csv(count_file)
    site_list = df_count['site'].tolist()
    site_list_merged = []
    for site in site_list:
        site_list_merged += [site]*20

    aa_list_merged = []
    for i in range(len(site_list)):
        aa_list_merged += AA

    rep=[i+1 for i in range(REP)]
    rep_list=[]
    for replicate in rep:
        rep_list.append('rep_'+str(replicate))
    df_pref_merged = pd.DataFrame(columns=['site', 'amino_acid'] + rep_list + ['average'])
    df_pref_merged['site'] = site_list_merged
    df_pref_merged['amino_acid'] = aa_list_merged

    for i in rep:
        df_pref = pd.read_csv(pref_list[i-1])
        df_pref_aa = df_pref[AA]
        df_pref_merged['rep_'+str(i)] = df_pref_aa.values.flatten()
    df_pref_merged['average'] = df_pref_merged[rep_list].mean(axis = 1)
    df_pref_merged.to_csv('./outputs/merged_preference/' + Target_name + '.csv.gz', index = False, compression = 'gzip')

REP=3
Target_name = 'PR8'
count_file = './data/raw_data/PR8_DNA_codoncounts.csv'
preference_merge(REP, Target_name, count_file)

# REP=2
# Target_name = 'Aichi68C'
# count_file = './data/raw_data/Aichi68C_DNA_codoncounts.csv'
# preference_merge(REP, Target_name, count_file)

# REP=3
# Target_name = 'HIV Env BG505'
# count_file = './data/raw_data/BG505_DNA_codoncounts.csv'
# preference_merge(REP, Target_name, count_file)

# REP=3
# Target_name = 'HIV Env BF520'
# count_file = './data/raw_data/BF520_DNA_codoncounts.csv'
# preference_merge(REP, Target_name, count_file)

# HIV Env BG505 FP16, FP20, VRC34

In [4]:
def transform_FP_VRC(index_file, pref_file, rep_num):
    df_index = pd.read_csv(index_file, delimiter = ',')
    df = pd.read_csv(pref_file)
    df = df.drop('wildtype', axis = 1)
    df = df.dropna()
    df['site'] = df['site'].astype('string')
    df_index['original'] = df_index['original'].astype('string')
    df_index['new'] = df_index['new'].astype('string')

    df['site'] = df['site'].map(df_index.set_index('new')['original'])
    df['site'] = df['site'].astype('int')
    df = df.rename(columns={'mutation': 'amino_acid', 'mutdiffsel': rep_num})
    return df

# FP16
index_file = './data/prefs/BG505_to_HXB2_numbering.txt'
Target_name = 'HIV bnAbs FP16'

pref_file = './data/prefs/FP16-02-500ug-rep-1.csv'
rep_num = 'rep_1'
df1 = transform_FP_VRC(index_file, pref_file, rep_num)

pref_file = './data/prefs/FP16-02-500ug-rep-3.csv'
rep_num = 'rep_2'
df2 = transform_FP_VRC(index_file, pref_file, rep_num)

df_merge = pd.merge(df1, df2, on = ['site', 'amino_acid'])
df_merge['average'] = df_merge[['rep_1', 'rep_2']].mean(axis=1)
df_merge = df_merge.sort_values('site')
df_merge.to_csv('./outputs/merged_preference/' + Target_name + '.csv.gz', index = False, compression = 'gzip')

#FP20
index_file = './data/prefs/BG505_to_HXB2_numbering.txt'
Target_name = 'HIV bnAbs FP20'

pref_file = './data/prefs/FP20-01-500ug-rep-1.csv'
rep_num = 'rep_1'
df1 = transform_FP_VRC(index_file, pref_file, rep_num)

pref_file = './data/prefs/FP20-01-500ug-rep-3.csv'
rep_num = 'rep_2'
df2 = transform_FP_VRC(index_file, pref_file, rep_num)

df_merge = pd.merge(df1, df2, on = ['site', 'amino_acid'])
df_merge['average'] = df_merge[['rep_1', 'rep_2']].mean(axis=1)
df_merge = df_merge.sort_values('site')
df_merge.to_csv('./outputs/merged_preference/' + Target_name + '.csv.gz', index = False, compression = 'gzip')

#VRC34
index_file = './data/prefs/BG505_to_HXB2_numbering.txt'
Target_name = 'HIV bnAbs VRC34'

pref_file = './data/prefs/VRC34-33ug-rep-1.csv'
rep_num = 'rep_1'
df1 = transform_FP_VRC(index_file, pref_file, rep_num)

pref_file = './data/prefs/VRC34-33ug-rep-3.csv'
rep_num = 'rep_2'
df2 = transform_FP_VRC(index_file, pref_file, rep_num)

df_merge = pd.merge(df1, df2, on = ['site', 'amino_acid'])
df_merge['average'] = df_merge[['rep_1', 'rep_2']].mean(axis=1)
df_merge = df_merge.sort_values('site')
df_merge.to_csv('./outputs/merged_preference/' + Target_name + '.csv.gz', index = False, compression = 'gzip')


# HIV Env BF520 human host, rhesus host

In [5]:
def transform_host(index_file, pref_file, rep_num):
    df_index = pd.read_csv(index_file, delimiter = ',')
    df = pd.read_csv(pref_file)
    df = df.dropna()
    df['site'] = df['site'].astype('string')
    df_index['original'] = df_index['original'].astype('string')
    df_index['new'] = df_index['new'].astype('string')

    df['site'] = df['site'].map(df_index.set_index('new')['original'])
    df['site'] = df['site'].astype('int')
    
    site_list = []
    AA_list = []
    df_pref_only = df[AA]
    for i in df['site'].tolist():
        site_list += [i]*20
        AA_list += AA
    df_pref = pd.DataFrame(columns=['site', 'amino_acid', rep_num]) 
    df_pref['site'] = site_list
    df_pref['amino_acid'] = AA_list
    df_pref[rep_num] = df_pref_only.values.flatten()
    return df_pref

# BF520 human host
index_file = './data/prefs/BF520c2_to_HXB2.csv'
Target_name = 'HIV BF520 human host'

pref_file = './data/prefs/HIV BF520 human host-1_prefs.csv'
rep_num = 'rep_1'
df1 = transform_host(index_file, pref_file, rep_num)

pref_file = './data/prefs/HIV BF520 human host-2_prefs.csv'
rep_num = 'rep_2'
df2 = transform_host(index_file, pref_file, rep_num)

df_merge = pd.merge(df1, df2, on = ['site', 'amino_acid'])
df_merge['average'] = df_merge[['rep_1', 'rep_2']].mean(axis=1)
df_merge = df_merge.sort_values('site')
df_merge.to_csv('./outputs/merged_preference/' + Target_name + '.csv.gz', index = False, compression = 'gzip')

# BF520 rhesus host
index_file = './data/prefs/BF520c2_to_HXB2.csv'
Target_name = 'HIV BF520 rhesus host'

pref_file = './data/prefs/HIV BF520 rhesus host-1_prefs.csv'
rep_num = 'rep_1'
df1 = transform_host(index_file, pref_file, rep_num)

pref_file = './data/prefs/HIV BF520 rhesus host-2_prefs.csv'
rep_num = 'rep_2'
df2 = transform_host(index_file, pref_file, rep_num)

df_merge = pd.merge(df1, df2, on = ['site', 'amino_acid'])
df_merge['average'] = df_merge[['rep_1', 'rep_2']].mean(axis=1)
df_merge = df_merge.sort_values('site')
df_merge.to_csv('./outputs/merged_preference/' + Target_name + '.csv.gz', index = False, compression = 'gzip')


In [ ]:
REP=3
Target_name = 'Matrix_M1'
count_file = './data/raw_data/HomM1_DNA_codoncounts.csv'
preference_merge(REP, Target_name, count_file)

In [ ]:
REP=3
Target_name = 'A549'
count_file = './data/raw_data/A549_DNA_codoncounts.csv'
preference_merge(REP, Target_name, count_file)

In [ ]:
REP=3
Target_name = 'CCL141'
count_file = './data/raw_data/CCL141_DNA_codoncounts.csv'
preference_merge(REP, Target_name, count_file)

In [ ]:
REP=2
Target_name = 'MS'
count_file = './data/raw_data/MS_DNA_codoncounts.csv'
preference_merge(REP, Target_name, count_file)

In [ ]:
REP=2
Target_name = 'MxA'
count_file = './data/raw_data/MxA_DNA_codoncounts.csv'
preference_merge(REP, Target_name, count_file)

In [ ]:
REP=2
Target_name = 'MxAneg'
count_file = './data/raw_data/MxAneg_DNA_codoncounts.csv'
preference_merge(REP, Target_name, count_file)

In [ ]:
REP=2
Target_name = 'HIV BF520 human host'
count_file = './data/raw_data/human_DNA_codoncounts.csv'
preference_merge(REP, Target_name, count_file)

REP=2
Target_name = 'HIV BF520 rhesus host'
count_file = './data/raw_data/rhesus_DNA_codoncounts.csv'
preference_merge(REP, Target_name, count_file)

In [ ]:
pd.read_csv('./outputs/selection_coefficients/HIV BF520 rhesus host.csv.gz')

In [ ]:
count_file = './data/raw_data/human_DNA_codoncounts.csv'
pd.read_csv(count_file)